In [ ]:
"""
    AveragedPowerspectrum{T<:AbstractFloat}

A type representing an averaged power spectrum computed from time series data.

# Fields
- `freqs::Vector{T}`: Frequency bins
- `power::Vector{T}`: Power values corresponding to each frequency bin
- `power_err::Vector{T}`: Error estimates for each power value
"""
struct AveragedPowerspectrum{T<:AbstractFloat}
    freqs::Vector{T}
    power::Vector{T}
    power_err::Vector{T}
end

"""
    from_events(times, dt, segment_size; norm="frac", use_common_mean=true, silent=false)

Compute an averaged power spectrum from an event list.

# Arguments
- `times`: Vector of event arrival times
- `dt`: Time resolution for binning
- `segment_size`: Size of segments to average (in seconds)
- `norm`: Normalization method ("frac", "leahy", "abs", or "none")
- `use_common_mean`: Whether to use common mean across all segments
- `silent`: Whether to suppress progress information

# Returns
- `AveragedPowerspectrum` object with frequencies, powers, and power errors
"""
function from_events(times, dt, segment_size; 
                    norm="frac", 
                    use_common_mean=true,
                    silent=false)
    
    # Get start and end times from the data
    tstart = minimum(times)
    tend = maximum(times)
    duration = tend - tstart
    
    # Create a simple 2D matrix for the GTI
    gti = [tstart tend]
    
    # Call the avg_pds_from_events function using the helpers
    result = avg_pds_from_events(
        times, 
        gti, 
        segment_size, 
        dt; 
        norm=norm, 
        use_common_mean=use_common_mean, 
        silent=silent
    )
    
    if isnothing(result)
        # Return an empty AveragedPowerspectrum if no valid segments
        return AveragedPowerspectrum{Float64}(Float64[], Float64[], Float64[])
    end
    
    # Extract frequencies and powers
    freqs = result.freq
    powers = result.power
    
    # Calculate power errors (approximation: power/sqrt(m))
    # Get the number of segments (m) from the metadata or estimate it
    n_segments = floor(Int, duration / segment_size)
    segments_used = min(n_segments, count(!isnothing, 
        get_flux_iterable_from_segments(times, gti, segment_size; n_bin=floor(Int, segment_size/dt))))
    
    if segments_used == 0
        segments_used = 1  # To avoid division by zero
    end
    
    power_err = powers ./ sqrt(segments_used)
    
    # Create and return the AveragedPowerspectrum
    return AveragedPowerspectrum(freqs, powers, power_err)
end

"""
    avg_pds_from_events(times, gti, segment_size, dt; norm="frac", use_common_mean=true, silent=false)

Computes an averaged power density spectrum from events.
This is a wrapper around the provided avg_pds_from_iterable function.
"""
function avg_pds_from_events(
    times::AbstractVector{<:Real}, 
    gti::AbstractMatrix{<:Real},
    segment_size::Real, 
    dt::Real; 
    norm::String="frac",
    use_common_mean::Bool=true, 
    silent::Bool=false, 
    fluxes=nothing, 
    errors=nothing)
    
    n_bin = round(Int, segment_size / dt)
    dt = segment_size / n_bin

    # Get flux segments as an iterable
    flux_iterable = get_flux_iterable_from_segments(
        times, gti, segment_size;
        n_bin=n_bin, fluxes=fluxes, errors=errors
    )
    
    # Process the iterable of segments
    result = avg_pds_from_iterable(
        flux_iterable, dt; 
        norm=norm,
        use_common_mean=use_common_mean,
        silent=silent
    )
    
    return result
end

"""
    avg_pds_from_iterable(flux_iterable, dt; norm="frac", use_common_mean=true, silent=false)

Calculates the averaged power spectrum from an iterable of flux segments.
Uses the provided helper functions for normalization and averaging.
"""
function avg_pds_from_iterable(flux_iterable, dt::Real; 
                              norm::String="frac", 
                              use_common_mean::Bool=true,
                              silent::Bool=false)
    # Progress tracking
    local_show_progress = identity
    
    # Initialize variables
    cross = nothing
    unnorm_cross = nothing
    n_ave = 0
    freq = nothing
    
    sum_of_photons = 0
    common_variance = nothing
    fgt0 = nothing
    n_bin = nothing
    
    # Process each flux segment
    for flux_tuple in local_show_progress(flux_iterable)
        if isnothing(flux_tuple) || all(iszero, flux_tuple.counts)
            continue
        end
        
        flux = flux_tuple.counts
        variance = nothing
        if hasproperty(flux_tuple, :errors)
            err = flux_tuple.errors
            variance = mean(err)^2
        end
        
        # Get the bin count and calculate FFT
        n_bin = length(flux)
        ft = fft(flux)
        n_ph = sum(flux)
        unnorm_power = real.(ft .* conj.(ft))
        
        # Update totals
        sum_of_photons += n_ph
        if !isnothing(variance)
            common_variance = sum_if_not_none_or_initialize(common_variance, variance)
        end
        
        # Initialize frequency array if needed
        if isnothing(cross)
            fgt0 = positive_fft_bins(n_bin)
            freq = fftfreq(n_bin, dt)[fgt0]
        end
        
        # Get positive frequencies
        unnorm_power_filtered = unnorm_power[fgt0]
        
        # Normalize if not using common mean
        cs_seg = copy(unnorm_power_filtered)
        if !use_common_mean
            mean_flux = n_ph / n_bin
            cs_seg = normalize_periodograms(
                unnorm_power_filtered, dt, n_bin; 
                mean_flux=mean_flux, n_ph=n_ph,
                norm=norm, variance=variance
            )
        end
        
        # Accumulate results
        cross = sum_if_not_none_or_initialize(cross, cs_seg)
        unnorm_cross = sum_if_not_none_or_initialize(unnorm_cross, unnorm_power_filtered)
        
        n_ave += 1
    end
    
    # If no valid segments, return nothing
    if isnothing(cross)
        return nothing
    end
    
    # Calculate averages
    n_ph = sum_of_photons / n_ave
    common_mean = n_ph / n_bin
    
    if !isnothing(common_variance)
        common_variance /= n_ave
    end
    
    # Average the accumulated powers
    unnorm_cross ./= n_ave
    cross ./= n_ave
    
    # Final normalization if using common mean
    if use_common_mean
        cross = normalize_periodograms(
            unnorm_cross, dt, n_bin; 
            mean_flux=common_mean, n_ph=n_ph,
            norm=norm, variance=common_variance
        )
    end
    
    # Create and return the results
    results = DataFrame()
    results.freq = freq
    results.power = cross
    results.unnorm_power = unnorm_cross
    
    return results
end

# Helper function - fftfreq implementation
function fftfreq(n, d=1.0)
    val = 1.0 / (n * d)
    results = zeros(n)
    N = (n-1) ÷ 2 + 1
    p1 = 0:N-1
    results[1:N] = p1
    p2 = -n ÷ 2:-1
    results[N+1:end] = p2
    return results * val
end